In [1]:
from glob import glob

In [2]:
import pandas as pd

In [3]:
files = sorted(glob("./*.xls*"))

In [4]:
for (i, n) in enumerate(files):
    print("{0:>3d} - {1}".format(i, n))

  0 - ./00_SocialCollective_Vernetzungsplattformen_151220.xls
  1 - ./Hamburger Flüchtlings- und Willkommensinitiativen.xlsx
  2 - ./InfoCompass – Content (1).xlsx
  3 - ./InfoCompass – Content (2).xlsx
  4 - ./InfoCompass – Content (3).xlsx
  5 - ./InfoCompass – Content.xlsx
  6 - ./MetaBrain_DB_TUB.xlsx
  7 - ./Online Information Services.xlsx
  8 - ./Refugee Crisis FB Groups & Refugee Maps Data.xlsx
  9 - ./Refugee Initiativen - Liste Wefugee.xlsx
 10 - ./refugeeboard_Frankfurt.xlsx
 11 - ./refugees.sahana.io_Organizations.xls


In [5]:
social_collective = pd.read_excel("00_SocialCollective_Vernetzungsplattformen_151220.xls")

In [6]:
hamburg = pd.read_excel("Hamburger Flüchtlings- und Willkommensinitiativen.xlsx", skiprows=19)

In [7]:
tub = pd.read_excel("MetaBrain_DB_TUB.xlsx", sheetname="DB")

In [8]:
ois = dict()
with pd.ExcelFile("Online Information Services.xlsx") as xls:
    ois_names = xls.sheet_names
    for name in ois_names:
        ois[name] = pd.read_excel(xls, name)

Skip Facebook groups for now (`files[4]`).

In [9]:
wefugee = dict()
with pd.ExcelFile("Refugee Initiativen - Liste Wefugee.xlsx") as xls:
    wefugee_names = xls.sheet_names
    for name in wefugee_names:
        wefugee[name] = pd.read_excel(xls, name)

In [10]:
refugeeboard = pd.read_excel("refugeeboard_Frankfurt.xlsx")

In [11]:
sahana = pd.read_excel("refugees.sahana.io_Organizations.xls")

Compare name and some kind of URL between all projects.

In [12]:
def simple_clean(df, header=["name", "url"]):
    df.columns = header
    return df[-(df[header[0]].isnull() | df[header[1]].isnull())]

In [13]:
data = list()
tmp = simple_clean(social_collective[["Projekt", "Webseite"]])
tmp["source"] = "social collective"
data.append(tmp)
tmp = simple_clean(tub[["title", "url"]])
tmp["source"] = "TU Berlin"
data.append(tmp)
for name in ois_names:
    tmp = simple_clean(ois[name][["Service Name", "Link"]])
    tmp["source"] = "OIS {}".format(name)
    data.append(tmp)
for name in wefugee_names:
    if name == "Ansprechpartner":
        continue
    tmp = simple_clean(wefugee[name][["Organization", "Link"]])
    tmp["source"] = "Wefugee {}".format(name)
    data.append(tmp)
tmp = simple_clean(refugeeboard[["name", "webseite"]])
tmp["source"] = "refugeeboard"
data.append(tmp)
tmp = simple_clean(sahana[["Name", "Website"]])
tmp["source"] = "Sahana"
data.append(tmp)

In [14]:
merge = pd.concat(data, ignore_index=True)

In [15]:
len(merge)

1001

In [16]:
merge.sort_values(["url", "source"], inplace=True)

In [17]:
duplicated_url = merge.duplicated(subset="url", keep=False)

In [18]:
duplicated_url.sum()

425

In [19]:
merge[duplicated_url]

,name,url,source
92,Adelante Antifaschistische Linke Bonn,http://adelante.blogsport.de,TU Berlin
662,Adelante Antifaschistische Linke Bonn,http://adelante.blogsport.de,Wefugee Bonn
105,alle-helfen-jetzt.de,http://alle-helfen-jetzt.de,TU Berlin
13,alle-helfen-jetzt.de,http://alle-helfen-jetzt.de,social collective
106,Allende 2 hilft,http://allende2hilft.de,TU Berlin
534,Allende 2 hilft,http://allende2hilft.de,Wefugee Berlin
111,Apps for Refugees,http://appsforrefugees.com,TU Berlin
19,Apps for Refugees,http://appsforrefugees.com,social collective
117,Arriving in Berlin,http://arriving-in-berlin.de/,TU Berlin
3,Arriving in Berlin,http://arriving-in-berlin.de/,social collective
